# Processing the TOPAZ MAGs DB published by [Alexander, et.al. 2021](https://www.biorxiv.org/content/10.1101/2021.07.25.453713v1):  
**Excerpt from abstract:**  
Here, we developed the EukHeist automated computational pipeline, to retrieve eukaryotic and prokaryotic metagenome assembled genomes (MAGs). We applied EukHeist to the eukaryote-dominated large-size fraction data (0.8-2000mm) from the Tara Oceans survey to recover both eukaryotic and prokaryotic MAGs, 10 which we refer to as TOPAZ (Tara Oceans Particle-AssociatedMAGs). The TOPAZMAGs consisted of more than 900 eukaryotic MAGs representing environmentally-relevant microbial and multicellular eukaryotes in addition to over 4,000 bacterial and archaeal MAGs. The bacterial and archaeal TOPAZ MAGs retrieved with EukHeist complement previous efforts by expanding the existing phylogenetic diversity through the increase in coverage of many likely particle- and host-associated taxa. We also demonstrate how the novel eukaryotic genomic content recovered from this study might be used to infer functional traits, such as trophic mode. By coupling MAGs and metatranscriptomic data, we explored ecologically-significant protistan groups, such as the Stramenopiles. A global survey of both eukaryotic and prokaryotic MAGs enabled the identification of ecological cohorts, driven by specific environmental factors, and putative host-microbe associations

### The DB is accessible from [TOPAZ MAGs](https://osf.io/gm564/).  

**The goal here is:**  
To properly format this protein seq DB in a way that one can use it with humann to profile environmental samples at the metabolic pathway level. To achieve this, we have to: 

    1. Scoop out sequences that can be taxonomically associated with an NCBI taxID down to the genus level, or at least the family level (using BioPython here!).
    2. Make sure that a given sequence is sufficiently large, e.g. seq-len > 50 AA.
    3. Annotate the target sequence against UniRef90_2020-2.
    4. Create a .dmnd DB out of this newly created .faa file of UniRef90 annotated sequences.
    
Jayson Gutierrez: 13/03/2022

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from subprocess import Popen, call, STDOUT, PIPE
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib
import json
import glob
import re
import gzip
import sys
import csv
import time
import io
import pathlib
from zipfile import ZipFile

import itertools

import json
import urllib.request

import Bio.SeqIO as bioseqio
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
#from Bio.Alphabet import IUPAC
from Bio import Entrez
from Bio import SeqIO

from ete3 import NCBITaxa
from taxonomy_ranks import TaxonomyRanks

from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

import urllib
from collections import OrderedDict 

import fastaparser
from io import StringIO

In [2]:
!which python

/home/VLIZ2000/jayson.gutierrez/anaconda3/envs/bioinformatics/bin/python


In [3]:
matplotlib.rcParams['savefig.dpi'] = 900
matplotlib.rcParams['figure.dpi'] = 600
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context("paper")
sns.set(font='serif')
# sns.set_style("white", {
#         "font.family": "serif",
#         "font.serif": ["Times", "Palatino", "serif"]
#     })
sns.set_style('ticks')

In [4]:
from utilities import *

In [176]:
def filter_mags(tax_str):
    '''Function to extract only desired taxonomic levels'''
    tax_lvls = np.array(tax_str.split(';')[::-1][:5])
    if(len(np.where(tax_lvls==';')[0])==0):
        s,g,f,o,c = tax_lvls
        return ("g__{}.s__{}|NCBI_taxID|c__{}.o__{}.f__{}".format(g,s,c,o,f))
    
def assign_lowest_NCBI_taxID(tax_lbl):
    '''Use function to assign NCBI tax ID at lowest level possible'''
    
    #Grab sps,genus lbls
    glbl,slbl = tax_lbl.split("|")[0].split(".s")
    slbl = "s" + slbl
    if(slbl == 's__-'):
        lw_tax_lbl = glbl.split("__")[-1]
    else:
        lw_tax_lbl = slbl.split("__")[-1]
    
    try:
        #Query tax IDs database
        rank_taxon = TaxonomyRanks(lw_tax_lbl)
        rank_taxon.get_lineage_taxids_and_taxanames()
        rank_dict =  list(rank_taxon.lineages.values())[0]
        
        #Fetch taxID
        if(rank_dict['species'][0]!='NA'):
            taxID = rank_dict['species'][1]
        elif(rank_dict['genus'][0]!='NA'):
            taxID = rank_dict['genus'][1]
        elif(rank_dict['family'][0]!='NA'):
            taxID = rank_dict['family'][1]
        elif(rank_dict['order'][0]!='NA'):
            taxID = rank_dict['order'][1]  
        elif(rank_dict['class'][0]!='NA'):
            taxID = rank_dict['class'][1]  
        
        #Set corresponding taxonomic lbls
        if(rank_dict['species'][0]=='NA'):
            sps_lbl = 'unclassified'
        else:
            sps_lbl = rank_dict['species'][0]

        if(rank_dict['genus'][0]=='NA'):
            gs_lbl = 'unclassified'
        else:
            gs_lbl = rank_dict['genus'][0]

        if(rank_dict['family'][0]=='NA'):
            fl_lbl = 'unclassified'
        else:
            fl_lbl = rank_dict['family'][0]

        if(rank_dict['order'][0]=='NA'):
            od_lbl = 'unclassified'
        else:
            od_lbl = rank_dict['order'][0]

        if(rank_dict['class'][0]=='NA'):
            cl_lbl = 'unclassified'
        else:
            cl_lbl = rank_dict['class'][0]

        return 'g__{}.s__{}|{}|c__{}.o__{}.f__{}'.format(sps_lbl,gs_lbl,taxID,cl_lbl,od_lbl,fl_lbl)        
        
    except:
        return 'NA'

#### Filtering dataframe: extracting MAGs that have been taxonomically assigned at least to the Class level

In [8]:
#Loading metadata for Euks
fid = 'data/TableS02_EukaryoticMAG.csv'
topaz_mags_meta_df = pd.read_csv(fid,index_col='Unnamed: 0')

In [98]:
#Identify SMAGs with desired taxonomic scope
tax_lbls = topaz_mags_meta_df[['eukulele_taxonomy','mmseqs_taxonomy']].apply(lambda l: ";".join([filter_mags(s) for s in l]), axis=1)
filtered_tax_lbls = tax_lbls[tax_lbls!='g__-.s__-|NCBI_taxID|c__-.o__-.f__-;g__-.s__-|NCBI_taxID|c__-.o__-.f__-']
filtered_tax_lbls2 = filtered_tax_lbls.apply(lambda s: sorted(s.split(";"), key = lambda s: s.count('-'))[0])
#Stringent filter: keep only those with at least genus label, species label even better
filtered_tax_lbls3 = filtered_tax_lbls2[~filtered_tax_lbls2.str.contains('g__-.s__-')]

Taxonomic affiliation is performed using the library: *taxonomy_ranks*, and example on how this works is shown below:

In [168]:
rank_taxon = TaxonomyRanks('Geminigera cryophila')
rank_taxon.get_lineage_taxids_and_taxanames()
list(rank_taxon.lineages.values())[0]

{'user_taxa': 'Geminigera cryophila',
 'taxa_searched': 'Geminigera cryophila',
 'superkingdom': ('Eukaryota', 2759),
 'kingdom': ('NA', 'NA'),
 'superphylum': ('NA', 'NA'),
 'phylum': ('NA', 'NA'),
 'subphylum': ('NA', 'NA'),
 'superclass': ('NA', 'NA'),
 'class': ('Cryptophyceae', 3027),
 'subclass': ('NA', 'NA'),
 'superorder': ('NA', 'NA'),
 'order': ('Pyrenomonadales', 589342),
 'suborder': ('NA', 'NA'),
 'superfamily': ('NA', 'NA'),
 'family': ('Geminigeraceae', 589343),
 'subfamily': ('NA', 'NA'),
 'genus': ('Geminigera', 46946),
 'subgenus': ('NA', 'NA'),
 'species': ('Geminigera cryophila', 46947)}

The following DF is the definitive list of MAGs filtered out from the TOPAZ DB, which will be used to further expand the first draft of our Marine Plankton Genomics DB suitable for metabolic pathway profiling using the HUMANnN pipeline.

In [191]:
filtered_mags_list = filtered_tax_lbls3.apply(assign_lowest_NCBI_taxID)
#Excluding the following samples for technical reasons
filtered_mags_list = filtered_mags_list[~filtered_mags_list.index.str.contains("TOPAZ_SPS2")]

Create definitive metadata DF and add the taxonomic labels created above, which are formatted to match HumanN

In [237]:
filtered_mags_definit_df = topaz_mags_meta_df.loc[filtered_mags_list.index]
filtered_mags_definit_df["smags_taxonomy_humann"] = filtered_mags_list
filtered_mags_definit_df.head()

,eukulele_taxonomy,mmseqs_taxonomy,groups,total_length,total_num_contigs,total_length_5kb,total_length_10kb,total_length_50kb,GC_percent,N50,...,num_unique_kegg,busco_eukaryota_odb10_completeness,busco_eukaryota_odb10_contamination,eukcc_completeness,eukcc_contamination,eukcc_lineage,ocean_region,depth,size_fraction,smags_taxonomy_humann
TOPAZ_IOD1_E001,Eukaryota;Hacrobia;Cryptophyta;Cryptophyceae;C...,Eukaryota;-;-;-;Cryptophyceae;-;Pyrenomonadale...,Cryptophyta,37074820,8377,15520088,2726308,0,58.27,4531,...,2922,57.25,2.75,69.57,8.70,Eukaryota_Fungi_Ascomycota,IO,DCM,0.8-5,g__Geminigera cryophila.s__Geminigera|46947|c_...
TOPAZ_IOD1_E003,Eukaryota;Hacrobia;Haptophyta;Prymnesiophyceae...,Eukaryota;-;Prymnesiophyceae;Haptista;Haptophy...,Haptophyta,30740252,7112,12314358,2329232,0,62.86,4424,...,2850,49.41,0.39,52.08,4.17,Eukaryota_Fungi_Microsporidia,IO,DCM,0.8-5,g__Chrysochromulina rotalis.s__Chrysochromulin...
TOPAZ_IOD1_E004,Eukaryota;Hacrobia;Haptophyta;Prymnesiophyceae...,Eukaryota;-;Prymnesiophyceae;Haptista;Haptophy...,Haptophyta,39191221,9102,15279501,2168595,0,62.08,4404,...,2859,50.98,0.39,NaN,NaN,NaN,IO,DCM,0.8-5,g__Chrysochromulina rotalis.s__Chrysochromulin...
TOPAZ_IOD1_E005,Eukaryota;Archaeplastida;Chlorophyta;Mamiellop...,Eukaryota;Viridiplantae;-;Chlorophyta;Mamiello...,Chlorophyta,17055374,1415,15579731,12443612,1136508,66.20,17298,...,3127,74.90,0.39,94.74,0.00,Eukaryota,IO,DCM,0.8-5,g__Micromonas sp..s__Micromonas|2812611|c__Mam...
TOPAZ_IOD1_E012,Eukaryota;Archaeplastida;Chlorophyta;Mamiellop...,Eukaryota;Viridiplantae;-;Chlorophyta;Mamiello...,Chlorophyta,12257231,1914,8691266,3943275,0,48.11,7330,...,2701,68.63,0.39,NaN,NaN,NaN,IO,DCM,0.8-5,g__Bathycoccus prasinos.s__Bathycoccus|41875|c...


Now let's loop over the entire set of SMAGs filetered above and pull out the corresponding sequences, reannotate the taxonomic label and add placeholders for functional annotation against UniRef90.  
**Uncomment to re-run code developed to pull out sequences**

In [244]:
# set_dir = '../marine_databases/TARA_Oceans/TOPAZ_MAGs'
# new_fasta_fid = os.path.join(set_dir,'Euk_TOPAZ_MAGs_v1_concat_filtered.faa')
# sample_ids_cont = []
# with ZipFile(os.path.join(set_dir,'Eukaryotic_TOPAZ_MAGs.zip')) as zf:
#     n=0
#     for sample_id in filtered_mags_definit_df.index:
#         try:
#             fid = '{0}/{0}.faa.gz'.format(sample_id)#'TOPAZ_SPS3_E081/TOPAZ_SPS3_E081.faa.gz'
#             tax_lbl_humann = filtered_mags_definit_df.loc[sample_id]['smags_taxonomy_humann']
#             with gzip.GzipFile(fileobj=zf.open(fid), mode='rb') as gf:
#                 for record in SeqIO.parse(StringIO(gf.read().decode("utf-8")), "fasta"):
#                     #Modify seq header
#                     seql = len(record)
#                     new_seq_header = "Seq{}-UniRef90|{}|{}".format(n,seql,tax_lbl_humann)
#                     record.id = ''
#                     record.name = ''
#                     record.description = ''
#                     record.id = new_seq_header

#                     #Dump record into file
#                     with open(new_fasta_fid, "a") as output_handle:
#                         bioseqio.write(record, output_handle, "fasta")          

#                     n+=1
#                 #Close current io stream
#                 fasta_io.close()
#                 sample_ids_cont.append(sample_id)
#         except:
#             pass

After looping over the TOPAZ collection of eukaryotic MAGs we end up with the following summary of the different organisms used as reference to extend our custom DB. It shall be noticed that different SMAgs with same taxonomic description might show some significant variation is gene content given that those genomes derived from different oceaninc pronvinces. So, it is worth considering all of them for mining for gene content. Significant gene sequence redundancy is taken care of downstream when mmseqs is applied on the final list of Diamond annotated sequences against UniRef90.

In [256]:
filtered_mags_definit_df2 = filtered_mags_definit_df.loc[sample_ids_cont]
no_mags_filtered = filtered_mags_definit_df2.shape[0]
print("No. of candidate MAGs from which proteins were pulled out: {}\n".format(no_mags_filtered))
filtered_mags_definit_df2['smags_taxonomy_humann'].value_counts()

No. of candidate MAGs from which proteins were pulled out: 495



g__Eurytemora affinis.s__Eurytemora|88015|c__Hexanauplia.o__Calanoida.f__Temoridae                                         150
g__Chrysochromulina rotalis.s__Chrysochromulina|412157|c__unclassified.o__Prymnesiales.f__Chrysochromulinaceae              31
g__Tigriopus californicus.s__Tigriopus|6832|c__Hexanauplia.o__Harpacticoida.f__Harpacticidae                                29
g__Bathycoccus prasinos.s__Bathycoccus|41875|c__Mamiellophyceae.o__Mamiellales.f__Bathycoccaceae                            22
g__Geminigera cryophila.s__Geminigera|46947|c__Cryptophyceae.o__Pyrenomonadales.f__Geminigeraceae                           21
                                                                                                                          ... 
g__Candida parapsilosis.s__Candida|5480|c__Saccharomycetes.o__Saccharomycetales.f__Debaryomycetaceae                         1
g__Stephanopyxis turris.s__Stephanopyxis|515487|c__Coscinodiscophyceae.o__Melosirales.f__Stephanopyxidaceae    

In [257]:
print("Total No. protein seqs pulled out from a list of {} MAGs".format(no_mags_filtered))
!grep -c '>' ../marine_databases/TARA_Oceans/TOPAZ_MAGs/Euk_TOPAZ_MAGs_v1_concat_filtered.faa

Total No. protein seqs pulled out from a list of 495 MAGs
4630020


### Processing output files from the functional annotation of the TOPAZ MAGs DB against the UniRef90 catalog via Diamond  
Diamond was run with the following settings --evalue 1e-3 --query-cover 80 --id 50

In [258]:
#Numeric columns in output files
new_col_names = ['Percentage_of_identical_matches', 'Alignment_length', 'Number_of_mismatches', 'Number_of_gap_openings',
                 'Start_of_alignment_in_query', 'End_of_alignment_in_query','Start_of_alignment_in_subject', 
                 'End_of_alignment_in_subject','Expected_value', 'Bit_score']

In [259]:
annotat_files = glob.glob('data/MAGs_filtered_chunk_annots_*.tsv')
sorted_annotat_files = sorted(annotat_files, key = lambda s: int(os.path.basename(s).split("_")[-1].replace(".tsv","")))

In [261]:
#Loop over all the annotation results files
annot_res_df_cont = []
for af in sorted_annotat_files:
    with open(af,"r") as annot_file:
        lines = annot_file.readlines()

    col_names = [s.replace(" ","_") for s in lines[2].split(":")[1].strip().split(', ')]

    #Fetch entries
    annot_res = []
    for l in lines[3:]:
        annot_res.append(l.strip().split('\t'))

    #Cast raw data into a DF
    annot_res_df = pd.DataFrame(data=annot_res,columns=col_names)

    #Convert fields to numeric and cast data as new DF
    new_annot_res_df = annot_res_df[new_col_names].apply(pd.to_numeric)
    new_annot_res_df["Subject_ID"] = annot_res_df["Subject_ID"]
    new_annot_res_df.index = annot_res_df["Query_ID"]
    #Stack up DFs
    annot_res_df_cont.append(new_annot_res_df)    

In [268]:
#Create full df of annotations
annot_all_results_df = pd.concat(annot_res_df_cont)
print("Total number of sequences: {}".format(annot_all_results_df.shape[0]))

Total number of sequences: 1672135


In [364]:
# #Set input/output files
# seq_hits_files = glob.glob('data/MAGs_filtered_chunk_annot_seqs_*.faa')
# sorted_seq_hits_files = sorted(seq_hits_files, key = lambda s: int(os.path.basename(s).split("_")[-1].replace(".faa","")))
# filtered_TOPAZ_MAGs_uniref90_annotat_fn = os.path.join(data_dir,"Euk_TOPAZ_MAGs_UniRef90_Annotated.faa")

# #Concatenate all the annotated fasta files -> dump into one single file
# concat_fasta = os.path.join(data_dir,"concat_MAGs_filtered_annot_seq_chunks.faa")
# concat_fasta_str = " ".join(sorted_seq_hits_files)
# cmd = "cat {} > {}".format(concat_fasta_str, concat_fasta)
# runShell(cmd,stdout=False)

Here we have to loop over the fasta file: concat_MAGs_filtered_annot_seq_chunks.faa and fetch the corresponding headers, which will be used for renaming the index of the DF annot_all_results_df, which was wrongly formatted internally by Diamond.

In [372]:
# correct_headers_cont = []
# with open(concat_fasta,"r") as handle:
#     for (n,record) in enumerate(bioseqio.parse(handle, "fasta")):
#         correct_headers_cont.append(record.description)

In [390]:
# #Update indexes of DF with correct headers
# annot_all_results_df.index = correct_headers_cont

In [388]:
# #Grab all the UniRef90 labels
# uniref90_lbls = annot_all_results_df['Subject_ID'].values
# tax_lbl_cont = []
# with open(concat_fasta,"r") as handle:
#     for (n,record) in enumerate(bioseqio.parse(handle, "fasta")):
#         ur90lbl = uniref90_lbls[n]
# #         if ("X" not in record.seq) and ('.f__' in record.id):
#         if ('.f__' in record.description):    
#             new_header = ur90lbl + record.description.split("UniRef90")[-1]
#             record.id = ''
#             record.name = ''
#             record.description = ''
#             record.id = new_header
#             record.name = new_header
#             record.id = new_header
#             #Dump record into file
#             with open(filtered_TOPAZ_MAGs_uniref90_annotat_fn, "a") as output_handle:
#                 bioseqio.write(record, output_handle, "fasta")
        
#         tax_lbl = "g{}".format(record.id.split("|g")[-1])
#         if (tax_lbl not in tax_lbl_cont) and ('.f__' in record.id):
#             tax_lbl_cont.append(tax_lbl)
                
# #Remove original fasta
# runShell("rm {}".format(concat_fasta),stdout=False)

In [394]:
# print("In the end, the total No. UniRef90 annotated sequences extracted from the TOPAZ catalog of MAGs is:")
# !grep -c '>' $filtered_TOPAZ_MAGs_uniref90_annotat_fn

In the end, the total No. UniRef90 annotated sequences extracted from the TOPAZ catalog of MAGs is:
1672135


In [397]:
# #Let's peek at the fasta file with the new set of protein sequences pulled out from TOPAZ, which were annotated against UniRef90
# !head $filtered_TOPAZ_MAGs_uniref90_annotat_fn

>UniRef90_L1JQR5|223|g__Geminigera cryophila.s__Geminigera|46947|c__Cryptophyceae.o__Pyrenomonadales.f__Geminigeraceae
MNQFNQMKVPKMPRGTIFTTAGGLALTVGGYVLVNHCMYNVEGGHRAVVYSRLSGMSQVV
KGEGTHFKLPWLQRPYIYNVRSTPRNIKSKSGSQDLQMVDINVRIIYRPNIEKLPEMYRS
LGLDYDERVLPSIANEVLKSVVAQYNAIDLLVKREQVSQQVRSRLEQRARDFYMVIDDVS
LTDIAFSPVFTESVEAKQVAQQEAERAKWVVDKALEEKKSIVI
>UniRef90_A0A6T7M7A7|377|g__Geminigera cryophila.s__Geminigera|46947|c__Cryptophyceae.o__Pyrenomonadales.f__Geminigeraceae
MAVVQSSGMAAGSPAVGGSHALVVSVGEHAPSASPGSAAPAHQAPCVPLWRARDMVLTEL
RALQQLAGQGRLEGKELLRLEVRVGGFGVDESMLWLRAQTMLPRVFFLNPDSSLLSAGCG
QAHLVSGDGDLPGELVKRGTPPEIRYFGGSRFDLGADVGQEWQDFQGHFFVLPQLEIVYM
PAPGQQPEHRDEWEQRSTGENKRSSLQNRAFTFAVNARAGAGEGWAGALQRAVQAVEGMR


In [395]:
# print("The above subset of annotated sequences comes from a total number of {} taxa".format(len(tax_lbl_cont)))

The above subset of annotated sequences comes from a total number of 67 taxa


### Merging together prok_euk_plankt_db_v1.0 (initial version of our custom DB) with the set of sequences pulled out from TOPAZ MAGs (filtered_TOPAZ_MAGs_uniref90_annotat_fn)

In [401]:
# custom_db_folder = '/home/VLIZ2000/jayson.gutierrez/meta_omics_data/marine_databases/CustomProkEukDB/'
# #Initially created DB containing a total of: 2131295 annotated seqs
# inital_db = os.path.join(custom_db_folder,"prok_euk_plankt_db_v1.0.faa")
# #Dump files into this new fasta file
# merged_fasta_fid = os.path.join(custom_db_folder,"prok_euk_plankt_db_v2.0.faa")
# #Merge
# cmd = "cat {} {} > {}".format(inital_db,filtered_TOPAZ_MAGs_uniref90_annotat_fn,merged_fasta_fid)
# runShell(cmd,stdout=False)

In [402]:
print("Checking total number of seqs in our upgraded custom DB:")
!grep -c '>' $merged_fasta_fid

Checking total number of seqs in our upgraded custom DB:
3803430


In [403]:
# print("Pushing our upgraded DB to the HPC")
# !scp $merged_fasta_fid vsc43582@login.hpc.ugent.be:/kyukon/data/gent/vo/001/gvo00125/vsc43582/Bioinformatics/HUMAnN3/data/databases/custom_db

Pushing our upgraded DB to the HPC
prok_euk_plankt_db_v2.0.faa                   100% 1725MB  42.5MB/s   00:40    
